In [1]:
from extract_text import TextExtractor

/home/exouser/miniconda3/envs/medsyn-3-8/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
impressions="right pleural effusion and large pancoast tumor in left lung"
output_folder = "/media/volume/gen-ai-volume/MedSyn/results/text_embed"  
file_name='test_k2.npy'
text_extractor = TextExtractor(resume_model="/media/volume/gen-ai-volume/MedSyn/models/test_run2")
text_extractor.run(impressions, output_folder, file_name)

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of CXRBertModel were not initialized from the model checkpoint at microsoft/BiomedVLP-CXR-BERT-specialized and are newly initialized: ['classifier.bias

<All keys matched successfully>
File already exists: /media/volume/gen-ai-volume/MedSyn/results/text_embed/test_k1.npy


In [3]:
from stage1 import run_diffusion_1
import accelerate
accelerate.state.AcceleratorState._shared_state.clear() # dirty hack to reset accelerator state
run_diffusion_1(input_folder="/media/volume/gen-ai-volume/MedSyn/results/text_embed", 
                output_folder= "/media/volume/gen-ai-volume/MedSyn/results/img_64_standard", 
                model_folder="/media/volume/gen-ai-volume/MedSyn/models/stage1", 
                num_sample=1,
                noise_folder="/media/volume/gen-ai-volume/MedSyn/results/img_64_standard/saved_noise/test_k2")

/home/exouser/miniconda3/envs/medsyn-3-8/lib/python3.8/site-packages/rotary_embedding_torch/rotary_embedding_torch.py:35: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/home/exouser/miniconda3/envs/medsyn-3-8/lib/python3.8/site-packages/rotary_embedding_torch/rotary_embedding_torch.py:254: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/home/exouser/MedsynBackend/src/stage1.py:1213: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only`

loading low-res model...
File already exists: /media/volume/gen-ai-volume/MedSyn/results/img_64_standard/dont_delete_sample_0.npy
Pre-saved noise not found! Generating new fixed noise instead.
Low Resolution: 100%|██████████| 51/51 [00:14<00:00,  3.47it/s]
ATTENTION PATH:  /media/volume/gen-ai-volume/MedSyn/results/img_64_standard/test_k1_sample_0_attention.npy


AttributeError: 'list' object has no attribute 'cpu'

In [5]:
from stage2 import run_diffusion_2
import accelerate
accelerate.state.AcceleratorState._shared_state.clear() # dirty hack to reset accelerator state

run_diffusion_2(input_folder="/media/volume/gen-ai-volume/MedSyn/results/img_64_standard", 
                        output_folder="/media/volume/gen-ai-volume/MedSyn/results/img_256_standard", 
                        model_folder="/media/volume/gen-ai-volume/MedSyn/models/stage2",
                        filename="test_k1.npy",
                        num_series_exists=0)

Number of parameters: 54307908
Map Location: cuda
Model path: /media/volume/gen-ai-volume/MedSyn/models/stage2/1000_ckpt/pytorch_model.bin


/home/exouser/MedsynBackend/src/stage2.py:1110: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(model_path, map_location=map_location), s

found 3 videos as gif files at /media/volume/gen-ai-volume/MedSyn/results/img_64_standard
loading model...
Load accelerator state: /media/volume/gen-ai-volume/MedSyn/models/stage2/1000_ckpt
training model...
num_samples: 1
High Resolution: 100%|██████████| 21/21 [01:26<00:00,  4.11s/it]
num_samples: 1
File already exists: /media/volume/gen-ai-volume/MedSyn/results/img_256_standard/dont_delete_sample_0.nii.gz
num_samples: 1
File already exists: /media/volume/gen-ai-volume/MedSyn/results/img_256_standard/20250205145212nopleurale_sample_0.nii.gz


## Visualizing the saliency map generations

In [17]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2

generated_filename = "test_k2_sample_0.npy"
output_folder = "/media/volume/gen-ai-volume/MedSyn/results/img_64_standard"
image_path = os.path.join(output_folder, generated_filename)

generated_image = np.load(image_path)  # Shape: (C, D, H, W) or (C, num_slices, height, width)

print(f"Loaded image shape: {generated_image.shape}")  # Check image dimensions


Loaded image shape: (1, 4, 64, 64, 64)


In [ ]:
# Define attention map path (modify filename if needed)
attention_map_filename = "20250205145212nopleurale_sample_0_attention.npy"  # Change if necessary
attention_map_path = os.path.join(output_folder, attention_map_filename)

# Load the attention maps
if os.path.exists(attention_map_path):
    attention_maps = np.load(attention_map_path)  # Shape: (num_heads, Slices, Height, Width)
    print(f"Loaded attention map shape: {attention_maps.shape}")
else:
    print("Attention maps not found! Make sure they were saved.")
    attention_maps = None


In [ ]:
def overlay_attention_on_slice(image_slice, attention_slice, alpha=0.5):
    """
    Overlay attention map on a single DICOM slice.
    - image_slice: (H, W) numpy array representing the DICOM slice.
    - attention_slice: (H, W) numpy array representing the attention heatmap.
    - alpha: blending ratio.
    """
    # Normalize image for visualization
    image_slice = (image_slice - np.min(image_slice)) / (np.max(image_slice) - np.min(image_slice))  # Normalize to [0,1]
    image_slice = (image_slice * 255).astype(np.uint8)  # Convert to [0,255] range
    
    # Normalize attention map
    attention_slice = (attention_slice - np.min(attention_slice)) / (np.max(attention_slice) - np.min(attention_slice))
    attention_slice = (attention_slice * 255).astype(np.uint8)  # Convert to [0,255]
    
    # Apply colormap to attention map
    heatmap = cv2.applyColorMap(attention_slice, cv2.COLORMAP_JET)
    
    # Blend original image and heatmap
    overlay = cv2.addWeighted(image_slice, 1 - alpha, heatmap, alpha, 0)
    
    return overlay

# Select a few sample slices to visualize
num_slices = generated_image.shape[1]  # Number of slices
selected_slices = [num_slices // 4, num_slices // 2, 3 * num_slices // 4]  # Select middle slices

plt.figure(figsize=(12, 6))
for i, slice_idx in enumerate(selected_slices):
    original_slice = generated_image[0, slice_idx]  # Extract the slice
    attention_slice = attention_maps[:, slice_idx].mean(axis=0)  # Average across attention heads
    
    # Overlay attention
    overlay = overlay_attention_on_slice(original_slice, attention_slice)

    plt.subplot(1, len(selected_slices), i+1)
    plt.imshow(overlay, cmap="gray")
    plt.title(f"Slice {slice_idx}")
    plt.axis("off")

plt.show()
